# Evaluation of local search without GNN

This notebook documents the performance of the initial local search algorithm for the MQCP on benchmark instances without the use of a GNN. 
Each instance is run for 60 seconds, results are compared to results from NuQCLq paper, Chen et al., 2021. 

In [20]:
using Graphs
using thesis.Instances
using thesis.LocalSearch
using DataFrames
using CSV

In [2]:
"""
    MQCPInstance

- `graph`: Input graph
- `target_γ`: Target density
- `best_known`: Size of best known solution

"""
struct MQCPInstance
    graph::SimpleGraph
    graph_id::String
    target_γ::Real
    best_known::Int

    function MQCPInstance(graph_id::String, target_γ::Real, best_known::Int)
        graph = load_instance("../inst/DIMACS/$graph_id.clq")
        new(graph, graph_id, target_γ, best_known)
    end
end

MQCPInstance

In [14]:
function obtain_settings(graph, stm_type, timelimit)
    construction_heuristic_settings = ConstructionHeuristicSettings(
                               0.4, # parameter p of exploration construction 
                               0.2, # GRASP parameter α
                               1,   # beamwidth β of initial construction
                               50,  # expansion_limit of initial construction
                               GreedyCompletionHeuristic() # guidance function of initial construction
                               )
    short_term_memory = stm_type(graph)
    settings = LocalSearchSettings(graph; 
                               construction_heuristic_settings, 
                               short_term_memory,
                               timelimit, 
                               max_iter=4000, 
                               next_improvement=true, 
                               )
    return settings
end

;

In [10]:
# selection of 20 benchmark instances
Instances = [
    MQCPInstance("brock400_1", 0.999,  27),
    MQCPInstance("brock400_2", 0.999,  29),
    MQCPInstance("brock400_2", 0.8  , 187),
    MQCPInstance("brock400_3", 0.999,  31),
    MQCPInstance("brock800_1", 0.9  ,  43),
    MQCPInstance("brock800_2", 0.999,  24),
    MQCPInstance("brock800_2", 0.8  ,  96),
    MQCPInstance("brock800_3", 0.999,  22),
    MQCPInstance("brock800_3", 0.9  ,  43),
    MQCPInstance("brock800_3", 0.8  ,  94),
    MQCPInstance("C1000.9",    0.999,  70),
    MQCPInstance("C1000.9",    0.95 , 222),
    MQCPInstance("C2000.9",    0.999,  82),
    MQCPInstance("C2000.9",    0.95 , 288),
    MQCPInstance("DSJC1000.5", 0.999, 15),
    MQCPInstance("DSJC1000.5", 0.8, 41),
    MQCPInstance("DSJC500.5", 0.999, 13),
    MQCPInstance("gen400_p0.9_55", 0.999, 55),
    MQCPInstance("gen400_p0.9_65", 0.999, 66),
    MQCPInstance("hamming10-4", 0.95, 88),
]
;

In [11]:
results_ts = []
for inst in Instances
    settings = obtain_settings(inst.graph, TabuList, 60.0)
    solution = run_MQCP(inst.graph, inst.target_γ; settings)
    push!(results_ts, length(solution))
end
;

In [12]:
results_cc = []
for inst in Instances
    settings = obtain_settings(inst.graph, ConfigurationChecking, 60.0)
    solution = run_MQCP(inst.graph, inst.target_γ; settings)
    push!(results_cc, length(solution))
end
;

In [13]:
df = DataFrame(GraphID=String[], V=Int[], E=Int[], Dens=Real[], γ=Real[], ConfChecking=Int[], TabuSearch=Int[], Best=Int[])

for (idx, inst) in enumerate(Instances)
    push!(df, (
        inst.graph_id,
        nv(inst.graph),
        ne(inst.graph),
        density(inst.graph),
        inst.target_γ,
        results_cc[idx],
        results_ts[idx],
        inst.best_known, 
        ))
end
df

,GraphID,V,E,Dens,γ,ConfChecking,TabuSearch,Best
,String,Int64,Int64,Real,Real,Int64,Int64,Int64
1,brock400_1,400,59723,0.748409,0.999,25,25,27
2,brock400_2,400,59786,0.749198,0.999,25,25,29
3,brock400_2,400,59786,0.749198,0.8,187,187,187
4,brock400_3,400,59681,0.747882,0.999,25,25,31
5,brock800_1,800,207505,0.649265,0.9,43,43,43
6,brock800_2,800,208166,0.651333,0.999,21,21,24
7,brock800_2,800,208166,0.651333,0.8,95,95,96
8,brock800_3,800,207333,0.648727,0.999,22,22,22
9,brock800_3,800,207333,0.648727,0.9,42,43,43


In [21]:
CSV.write("df_60.csv", df)

"df_60.csv"

In [15]:
results_ts_300 = []
for inst in Instances
    settings = obtain_settings(inst.graph, TabuList, 300.0)
    solution = run_MQCP(inst.graph, inst.target_γ; settings)
    push!(results_ts_300, length(solution))
end
;

In [16]:
results_cc_300 = []
for inst in Instances
    settings = obtain_settings(inst.graph, ConfigurationChecking, 300.0)
    solution = run_MQCP(inst.graph, inst.target_γ; settings)
    push!(results_cc_300, length(solution))
end
;

In [17]:
df_300 = DataFrame(GraphID=String[], V=Int[], E=Int[], Dens=Real[], γ=Real[], ConfChecking=Int[], TabuSearch=Int[], Best=Int[])

for (idx, inst) in enumerate(Instances)
    push!(df_300, (
        inst.graph_id,
        nv(inst.graph),
        ne(inst.graph),
        density(inst.graph),
        inst.target_γ,
        results_cc_300[idx],
        results_ts_300[idx],
        inst.best_known, 
        ))
end
df_300

,GraphID,V,E,Dens,γ,ConfChecking,TabuSearch,Best
,String,Int64,Int64,Real,Real,Int64,Int64,Int64
1,brock400_1,400,59723,0.748409,0.999,25,25,27
2,brock400_2,400,59786,0.749198,0.999,27,25,29
3,brock400_2,400,59786,0.749198,0.8,187,187,187
4,brock400_3,400,59681,0.747882,0.999,27,25,31
5,brock800_1,800,207505,0.649265,0.9,43,43,43
6,brock800_2,800,208166,0.651333,0.999,21,21,24
7,brock800_2,800,208166,0.651333,0.8,95,95,96
8,brock800_3,800,207333,0.648727,0.999,22,22,22
9,brock800_3,800,207333,0.648727,0.9,43,43,43


In [22]:
CSV.write("df_300.csv", df)

"df_300.csv"